In [ ]:
import face_recognition
import cv2
import pandas as pd
import numpy as np

# Charger le dataset
data = pd.read_csv('Classeur1.csv', delimiter=';')  # CSV avec les colonnes: first_name, last_name, image_path

# Encoder les visages
known_face_encodings = []
known_face_names = []

for index, row in data.iterrows():
    image = face_recognition.load_image_file(row['image_path'])
    face_encodings = face_recognition.face_encodings(image)
    if face_encodings:
        known_face_encodings.append(face_encodings[0])
        known_face_names.append(f"{row['first_name']} {row['last_name']}")
    else:
        print(f"Aucun visage trouvé dans l'image : {row['image_path']}")

# Ouvrir la caméra vidéo
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convertir le frame en RGB (OpenCV charge en BGR)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Appliquer un flou gaussien pour débruiter l'image
    blurred_frame = cv2.GaussianBlur(rgb_frame, (5, 5), 0)

    # Trouver les emplacements des visages dans l'image
    face_locations = face_recognition.face_locations(blurred_frame)

    # Vérifier qu'il y a des visages détectés
    if face_locations:
        # Obtenir les encodages des visages
        face_encodings = face_recognition.face_encodings(blurred_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Inconnu"

            # Calculer les distances entre les visages
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)

            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Dessiner un rectangle autour du visage et afficher le nom
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

    # Afficher la vidéo avec les visages détectés
    cv2.imshow('Video', frame)

    # Quitter la capture vidéo en appuyant sur 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la capture vidéo et fermer la fenêtre
video_capture.release()
cv2.destroyAllWindows()
